In [58]:
import qflib as qf
import numpy as np
import pandas as pd

qf.version()

'0.7.0-debug'

### Problem 1

In [59]:
maturities = [1/12, 1/4, 1/2, 3/4, 1, 2, 3, 5]
spot_rates = [0.01, 0.02, 0.03, 0.035, 0.04, 0.0475, 0.0525, 0.06]

yc_name = "USD-LIBOR"
qf.ycCreate(yc_name, maturities, spot_rates, 1)

strike_rate = 0.05
coupon_rate = 0.05
fwd_rate_vol = 0.2
notional = 1000000
tenor = 0.5

reset_times = np.arange(0.5, 3, tenor)
payment_times = reset_times + tenor

total_caplet_price = 0.0
for i in range(len(reset_times)):
    caplet_price = qf.capFloorletBS(1, yc_name, strike_rate, reset_times[i], tenor, fwd_rate_vol)
    total_caplet_price += caplet_price
notional_caplet = total_caplet_price * notional

print(f"Part 1: The total price of the 5 caplets are: ${notional_caplet}")

total_floorlet_price = 0.0
for i in range(len(reset_times)):
    floorlet_price = qf.capFloorletBS(-1, yc_name, strike_rate, reset_times[i], tenor, fwd_rate_vol)
    total_floorlet_price += floorlet_price
notional_floorlet = total_floorlet_price * notional

print(f"Part 2: The total price of the 5 floorlets are: ${notional_floorlet}")

pv_fixed_leg = 0.0
for i in range(len(payment_times)):
    fixed_payment = coupon_rate * notional * tenor
    df = qf.discount(yc_name, payment_times[i])
    pv_payment = fixed_payment * df
    pv_fixed_leg += pv_payment

pv_floating_leg = 0.0
for i in range(len(payment_times)):
    fwd_rate_cont = qf.fwdRate(yc_name, reset_times[i], payment_times[i])
    fwd_rate_disc = qf.fromContCmpd(fwd_rate_cont, int(1/tenor))
    floating_payment = fwd_rate_disc * notional * tenor
    df = qf.discount(yc_name, payment_times[i])
    pv_payment = floating_payment * df
    pv_floating_leg += pv_payment

swap_value = pv_floating_leg - pv_fixed_leg

print(f"Part 3: The value of the swap is: ${swap_value}")

print(f"Part 4:\nThe total caplet price minus total floorlet price is\n{notional_caplet:.4f} - {notional_floorlet:.4f} = {(notional_caplet - notional_floorlet):.4f}\nwhich is equivalent to the value of the swap {swap_value:.4f}.\nSo the parity relation CAP - FLOOR = SWAP holds true.")

Part 1: The total price of the 5 caplets are: $10244.265626430695
Part 2: The total price of the 5 floorlets are: $15101.817906797192
Part 3: The value of the swap is: $-4857.552280366464
Part 4:
The total caplet price minus total floorlet price is
10244.2656 - 15101.8179 = -4857.5523
which is equivalent to the value of the swap -4857.5523.
So the parity relation CAP - FLOOR = SWAP holds true.


### Problem 2

In [65]:
credit_spread = 0.01
cds_rate_cont = 0.01
recovery = 0.0
freq = 2
notional = 1000000

cds_rate_disc = qf.fromContCmpd(cds_rate_cont, freq)
maturities = [1, 2, 3, 4, 5]

default_legs = []
premium_legs = []
for i in range(len(maturities)):
    pv_cds = qf.cdsPV(yc_name, credit_spread, cds_rate_disc, recovery, maturities[i], freq)
    default_legs.append(pv_cds[0] * notional)
    premium_legs.append(pv_cds[1] * notional)

difference = np.array(default_legs) - np.array(premium_legs)

table = {
    "Maturity": maturities,
    "Default Leg": default_legs,
    "Premium Leg": premium_legs,
    "Difference": difference.round(8)
}
df = pd.DataFrame(table)

print(df)
print(f"For every maturity, the difference between the default leg and premium leg is 0")






   Maturity   Default Leg   Premium Leg  Difference
0         1   9743.642043   9743.642043         0.0
1         2  18966.157115  18966.157115         0.0
2         3  27640.907344  27640.907344         0.0
3         4  35744.622236  35744.622236         0.0
4         5  43300.475919  43300.475919         0.0
For every maturity, the difference between the default leg and premium leg is 0
